In [ ]:
!du -h --max-depth=1 "/content/drive/My Drive/nsd_data"

20K	/content/drive/My Drive/nsd_data/algonauts_2023
24K	/content/drive/My Drive/nsd_data


In [ ]:
import os
!git clone https://github.com/childmindresearch/algonauts23 algonauts_2023
%cd algonauts_2023


Cloning into 'algonauts_2023'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 52 (delta 6), reused 49 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 114.17 KiB | 3.94 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/algonauts_2023


In [ ]:
%ls
%cd dataset

algonauts23/  dataset/  pyproject.toml  requirements.txt  setup.cfg
config/       LICENSE   README.md       scripts/          tests/
/content/drive/MyDrive/nsd_data/algonauts_2023/dataset


In [ ]:
!pip install datasets==2.12.0


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00


In [ ]:
!gdown --folder --output /content/drive/MyDrive/nsd_data/ 1sgGqbGTqbvmq7zzznjbansv728kzgE-l

Retrieving folder contents
Retrieving folder 1BPF8J-CiXz7r6Mf3AlpAzSj6f47L9FIT algonauts_2023
Retrieving folder 1-E0m53jynpY6XBsiZ4VYfKJ70gfdw_eW derived_splits
Retrieving folder 1-OABF4__af1fnKd-v7zeMgp07eXmLV-1 subj01
Retrieving folder 1-Ng5b3t8sQRICy7Bh6hWxYI0ZU6s5mHJ subj02
Retrieving folder 1-F2GWeuBg3OvD8O9BvQYuuh35pU8VPXE subj03
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Download completed


In [ ]:
!pip install gdown

import os

output_dir = "/content/drive/MyDrive/nsd_data/algonauts_2023_challenge_data"
file_ids = {
    "subj01": "16BjZM192Go2pkufcxr51lxBHdJfKYaMt",
    "subj02": "1G-swRizJR6Xya8kqahKNEh8HMdG7PZV-",
    "subj03": "1rtUdnYqEQR_Usw05zfOkOIDjXLbe2WAt",
}

for subj, file_id in file_ids.items():
    print(f"Downloading {subj}...")
    zip_path = os.path.join(output_dir, f"{subj}.zip")
    os.makedirs(output_dir, exist_ok=True)
    os.system(f"gdown --id {file_id} --output {zip_path}")

print("Unzipping downloaded files...")
for subj in file_ids.keys():
    zip_path = os.path.join(output_dir, f"{subj}.zip")
    os.system(f"unzip {zip_path} -d {output_dir}")
    os.remove(zip_path)


Unzipping downloaded files...


In [ ]:
import logging
import shutil
from pathlib import Path
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

SUBS = ['subj01', 'subj02', 'subj03']

def seed_hash(*args):
    return hash(args) % (2**32)

ROOT = Path("/content/drive/MyDrive/nsd_data")
PARENT_SPLIT = "training-official"
DERIVED_SPLITS = {"train": 0.85, "val": 0.1, "testval": 0.05}
SEED = 2023

# Logging configuration
logging.basicConfig(
    format="[%(levelname)s %(asctime)s]: %(message)s",
    level=logging.INFO,
    datefmt="%y-%m-%d %H:%M:%S",
)

# Function to generate data splits
def generate_splits(num_samples, split_sizes, seed):
    """
    Generate reproducible data splits.
    """
    assert sum(split_sizes) == 1.0, "split_sizes must sum to 1"

    split_lengths = np.asarray(split_sizes) * num_samples
    split_ends = np.round(np.cumsum(split_lengths)).astype(int)
    split_starts = np.concatenate([[0], split_ends[:-1]])

    rng = np.random.default_rng(seed)
    indices = rng.permutation(num_samples)

    splits = [
        np.sort(indices[start:end]) for start, end in zip(split_starts, split_ends)
    ]
    return splits

challenge_data_dir = ROOT / "algonauts_2023_challenge_data"
out_dir = ROOT / "algonauts_2023" / "derived_splits"

if not ROOT.exists():
    ROOT.mkdir(parents=True, exist_ok=True)

if out_dir.exists():
    shutil.rmtree(out_dir)
out_dir.mkdir(parents=True, exist_ok=True)

# Process subj01, subj02, and subj03
for sub in SUBS:
    sub_dir = challenge_data_dir / sub
    sub_out_dir = out_dir / sub
    sub_out_dir.mkdir(parents=True, exist_ok=True)

    # Load the training fMRI data
    train_fmri_path = sub_dir / "training_split" / "training_fmri" / "lh_training_fmri.npy"
    if train_fmri_path.exists():
        train_fmri = np.load(train_fmri_path, mmap_mode="r")
        num_samples = len(train_fmri)

        # Generate splits
        seed = seed_hash("generate_splits", PARENT_SPLIT, sub, SEED)
        sizes = list(DERIVED_SPLITS.values())
        indices = generate_splits(num_samples, sizes, seed)

        # Save split indices
        for split, ind in zip(DERIVED_SPLITS, indices):
            logging.info("%s/%s: %s", sub, split, ind[:10])
            np.save(sub_out_dir / f"{split}_indices.npy", ind)
    else:
        logging.warning("File not found for %s: %s", sub, train_fmri_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls -r /content/drive/MyDrive/nsd_data/

algonauts_2023_challenge_data  algonauts_2023


In [ ]:
!pip install datasets pillow tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import logging
import shutil
from pathlib import Path
import numpy as np
from datasets import Dataset, Image
from PIL.Image import Resampling
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

ROOT = Path("/content/drive/MyDrive/nsd_data/algonauts_2023")
SEED = 2023
PARENT_SPLITS = {"train": "training", "val": "training", "testval": "training", "test": "test"}
SUBS = ['subj01', 'subj02', 'subj03']

logging.basicConfig(format="[%(levelname)s %(asctime)s]: %(message)s",
                    level=logging.INFO,
                    datefmt="%y-%m-%d %H:%M:%S")

def generate_dataset(root: Path, split: str = "train"):
    algonauts_dir = root.parent / "algonauts_2023_challenge_data"
    parent_split = PARENT_SPLITS[split]

    for sub_id, sub in enumerate(SUBS):
        logging.info("Generating data for subject %s", sub)
        image_list = load_image_list(algonauts_dir, sub, parent_split)
        logging.info("Num images: %d", len(image_list))

        if parent_split != "test":
            fmri = load_fmri(algonauts_dir, sub, parent_split)
            logging.info("fMRI shape: %s", fmri.shape)
        else:
            fmri = None

        indices = load_indices(root, sub, split)
        if indices is None:
            indices = np.arange(len(image_list))

        for idx in tqdm(indices):
            path = image_list[idx]
            entities = dict([part.split("-", maxsplit=2) for part in path.stem.split("_")])
            nsd_id = int(entities["nsd"])

            activity = fmri[idx] if fmri is not None else None
            yield {"split": split, "subject": sub, "subject_id": sub_id,
                   "sample_id": idx, "nsd_id": nsd_id, "image": str(path.absolute()), "activity": activity}


def load_image_list(algonauts_dir, sub, parent_split):
    return sorted((algonauts_dir / sub / f"{parent_split}_split" / f"{parent_split}_images").glob("*.png"))

def load_fmri(algonauts_dir, sub, parent_split):
    fmri_dir = algonauts_dir / sub / f"{parent_split}_split" / f"{parent_split}_fmri"
    fmri_lh = np.load(fmri_dir / f"lh_{parent_split}_fmri.npy")
    fmri_rh = np.load(fmri_dir / f"rh_{parent_split}_fmri.npy")
    return np.concatenate([fmri_lh, fmri_rh], axis=1)

def load_indices(root, sub, split):
    indices_path = root / "derived_splits" / sub / f"{split}_indices.npy"
    return np.load(indices_path) if indices_path.exists() else None

def get_transforms(img_size):
    def transforms(examples):
        examples["image"] = [image.resize((img_size, img_size), resample=Resampling.BICUBIC)
                             for image in examples["image"]]
        return examples
    return transforms

splits = ["train", "val", "testval"]
img_size = 256  # Resize all images to 256x256
workers = 4

import datasets
import os

# Disable Hugging Face caching
os.environ["HF_DATASETS_CACHE"] = "/dev/null"
datasets.config.HF_DATASETS_CACHE = "/tmp/hf_datasets_cache"

for split in splits:
    logging.info(f"Processing split: {split}")
    dset = Dataset.from_generator(generate_dataset, gen_kwargs={"root": ROOT, "split": split})
    dset = dset.cast_column("image", Image())

    if img_size > 0:
        dset = dset.map(get_transforms(img_size), batched=True, batch_size=256, num_proc=workers)

    if split == "train":
        dset = dset.shuffle(seed=SEED)

    # Save the processed dataset
    out_dir = ROOT / "processed" / f"size-{img_size}" / split
    if out_dir.exists():
        shutil.rmtree(out_dir)
    dset.save_to_disk(out_dir)
    logging.info(f"Saved {split} dataset to {out_dir}")




Mounted at /content/drive


Generating train split: 0 examples [00:00, ? examples/s]


100%|██████████| 8365/8365 [00:09<00:00, 841.81it/s]

100%|██████████| 8365/8365 [00:12<00:00, 696.20it/s]

100%|██████████| 7720/7720 [00:13<00:00, 586.87it/s]


Map (num_proc=4):   0%|          | 0/24450 [00:00<?, ? examples/s]

Saving the dataset (0/14 shards):   0%|          | 0/24450 [00:00<?, ? examples/s]

...

Saving the dataset (0/2 shards):   0%|          | 0/2876 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]


100%|██████████| 492/492 [00:00<00:00, 27837.55it/s]

100%|██████████| 492/492 [00:00<00:00, 30997.05it/s]

100%|██████████| 454/454 [00:01<00:00, 267.26it/s]


Map (num_proc=4):   0%|          | 0/1438 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1438 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is NOT available.")


CUDA is available!
Number of GPUs: 1
GPU 0: NVIDIA A100-SXM4-40GB


In [ ]:
!pip install datasets pillow tqdm
from datasets import load_from_disk

# Load the train, validation, and testval splits
train_dataset = load_from_disk('/content/drive/MyDrive/nsd_data/algonauts_2023/processed/size-256/train')
val_dataset = load_from_disk('/content/drive/MyDrive/nsd_data/algonauts_2023/processed/size-256/val')
testval_dataset = load_from_disk('/content/drive/MyDrive/nsd_data/algonauts_2023/processed/size-256/testval')

print(train_dataset)
print(train_dataset[0])


Dataset({
    features: ['split', 'subject', 'subject_id', 'sample_id', 'nsd_id', 'image', 'activity'],
    num_rows: 24450
})
{'split': 'train', 'subject': 'subj01', 'subject_id': 0, 'sample_id': 1352, 'nsd_id': 10459, 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x256 at 0x798C98794430>, 'activity': [-0.6802130937576294, -0.03223996236920357, -0.28753116726875305, -1.3810890913009644, -0.8335912227630615, -0.9322627782821655, -0.24177655577659607, -0.21463702619075775, 0.1585964858531952, 0.18870893120765686, 0.6244354248046875, 1.6187149286270142, -0.698640763759613, -0.4649065434932709, -1.1286858320236206, -1.2322875261306763, -0.6444418430328369, 0.8209864497184753, -0.5930544137954712, -0.629973292350769, 0.3073829114437103, -0.7263011336326599, -1.1012169122695923, 0.3777778744697571, -0.18045669794082642, -0.2009589821100235, -0.8423824310302734, 0.0539160817861557, -1.6119840145111084, -1.8683416843414307, 0.17024774849414825, -0.6863701343536377, -0.256861

In [ ]:
!pip install transformers
import torch
import torch.nn as nn
from transformers import ViTModel

class VAE(nn.Module):
    def __init__(self, latent_dim=256):
        super(VAE, self).__init__()

        # ViT Encoder
        self.encoder = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
        self.latent_dim = latent_dim

        # Fully connected layers for latent space
        self.fc_mu = nn.Linear(self.encoder.config.hidden_size, latent_dim)
        self.fc_logvar = nn.Linear(self.encoder.config.hidden_size, latent_dim)

        self.decoder_input = nn.Linear(latent_dim, 512 * 7 * 7)

        # Decoder
        self.decoder = nn.Sequential(
          nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1), # 7x7 -> 14x14
          nn.ReLU(),
          nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1), # 14x14 -> 28x28
          nn.ReLU(),
          nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # 28x28 -> 56x56
          nn.ReLU(),
          nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),   # 56x56 -> 112x112
          nn.ReLU(),
          nn.ConvTranspose2d(32, 3, kernel_size=3, stride=2, padding=1, output_padding=1),    # 112x112 -> 224x224
          nn.Sigmoid()
)


    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # Encoder: ViT
        vit_outputs = self.encoder(x, output_hidden_states=False)
        hidden_states = vit_outputs.last_hidden_state[:, 0]  # Use CLS token

        # Latent space
        mu = self.fc_mu(hidden_states)
        logvar = self.fc_logvar(hidden_states)
        z = self.reparameterize(mu, logvar)

        # Decoder
        recon = self.decoder_input(z)
        recon = recon.view(-1, 512, 7, 7)
        recon = self.decoder(recon)
        return recon, mu, logvar


In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# PyTorch Dataset
class FMRIImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = sample['image']
        activity = torch.tensor(sample['activity'], dtype=torch.float32)  # fMRI signal

        if self.transform:
            image = self.transform(image)

        return image, activity

# Wrap dataset with PyTorch
train_data = FMRIImageDataset(train_dataset, transform)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)


In [ ]:
import torch.optim as optim

def loss_function(recon_x, x, mu, logvar):
    # Reconstruction loss (MSE)
    recon_loss = nn.functional.mse_loss(recon_x, x, reduction='sum')
    # KL divergence
    kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kld_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = VAE().to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-4)

save_path = '/content/drive/MyDrive/vae_model'
os.makedirs(save_path, exist_ok=True)

# Training loop
num_epochs = 20
print("Starting training!")
for epoch in range(num_epochs):
    print("Epoch " + str(epoch))
    vae.train()
    train_loss = 0
    for images, _ in train_loader:
        images = images.to(device)

        optimizer.zero_grad()
        recon_images, mu, logvar = vae(images)
        loss = loss_function(recon_images, images, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss / len(train_loader.dataset):.4f}")
    torch.save(vae.state_dict(), os.path.join(save_path, f"vae_epoch_{epoch+1}.pth"))
    print(f"Model saved to {os.path.join(save_path, f'vae_epoch_{epoch+1}.pth')}")

Starting training!
Epoch 0
Epoch [1/20], Loss: 7447.9807
Model saved to /content/drive/MyDrive/vae_model/vae_epoch_1.pth
Epoch 1
Epoch [2/20], Loss: 5307.5163
Model saved to /content/drive/MyDrive/vae_model/vae_epoch_2.pth
Epoch 2
Epoch [3/20], Loss: 4313.4078
Model saved to /content/drive/MyDrive/vae_model/vae_epoch_3.pth
Epoch 3
Epoch [4/20], Loss: 3865.7693
Model saved to /content/drive/MyDrive/vae_model/vae_epoch_4.pth
Epoch 4
Epoch [5/20], Loss: 3567.0830
Model saved to /content/drive/MyDrive/vae_model/vae_epoch_5.pth
Epoch 5
Epoch [6/20], Loss: 3335.8230
Model saved to /content/drive/MyDrive/vae_model/vae_epoch_6.pth
Epoch 6
Epoch [7/20], Loss: 3174.2204
Model saved to /content/drive/MyDrive/vae_model/vae_epoch_7.pth
Epoch 7
Epoch [8/20], Loss: 3040.1079
Model saved to /content/drive/MyDrive/vae_model/vae_epoch_8.pth
Epoch 8
Epoch [9/20], Loss: 2933.4372
Model saved to /content/drive/MyDrive/vae_model/vae_epoch_9.pth
Epoch 9
Epoch [10/20], Loss: 2842.5431
Model saved to /content/

In [ ]:
import torch
import os

save_path = '/content/drive/MyDrive/vae_model'
model_file = os.path.join(save_path, 'vae_epoch_20.pth')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = VAE().to(device)

# Load the state dictionary
vae.load_state_dict(torch.load(model_file, map_location=device))
vae.eval()

print(f"Model loaded successfully from {model_file}")

import matplotlib.pyplot as plt

images, _ = next(iter(train_loader))
images = images.to(device)

with torch.no_grad():
    recon_images, _, _ = vae(images)

# Denormalize
def denormalize(tensor):
    return tensor * 0.5 + 0.5

images = denormalize(images.cpu())
recon_images = torch.clamp(denormalize(recon_images.cpu()), 0, 1)

# Visualize original and reconstructed images
fig, axes = plt.subplots(2, 8, figsize=(16, 4))
for i in range(8):
    axes[0, i].imshow(images[i].permute(1, 2, 0))
    axes[0, i].axis('off')
    axes[1, i].imshow(recon_images[i].permute(1, 2, 0))
    axes[1, i].axis('off')

axes[0, 0].set_title('Original Images')
axes[1, 0].set_title('Reconstructed Images')
plt.show()


### **fMRI Integration with VAE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets pillow tqdm transformers

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from datasets import load_from_disk
from transformers import ViTModel
import torch.nn.functional as F

# Load datasets
train_dataset = load_from_disk('/content/drive/MyDrive/nsd_data/algonauts_2023/processed/size-256/train')
val_dataset = load_from_disk('/content/drive/MyDrive/nsd_data/algonauts_2023/processed/size-256/val')
testval_dataset = load_from_disk('/content/drive/MyDrive/nsd_data/algonauts_2023/processed/size-256/testval')

print(train_dataset)
print(train_dataset[0])
fmri_example = train_dataset[0]['activity']
fmri_dim = len(fmri_example)

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

class FMRIImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = sample['image']
        activity = torch.tensor(sample['activity'], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)
        return image, activity

train_data = FMRIImageDataset(train_dataset, transform)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

class VAE(nn.Module):
    def __init__(self, latent_dim=256, fmri_dim=None):
        super(VAE, self).__init__()
        if fmri_dim is None:
            raise ValueError("fmri_dim must be specified")

        # ViT Encoder
        self.encoder = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
        self.latent_dim = latent_dim

        # For image-derived latent
        self.fc_mu = nn.Linear(self.encoder.config.hidden_size, latent_dim)
        self.fc_logvar = nn.Linear(self.encoder.config.hidden_size, latent_dim)

        # For fMRI-derived latent (fMRI -> latent)
        self.fmri_to_latent = nn.Sequential(
            nn.Linear(fmri_dim, 512),
            nn.ReLU(),
            nn.Linear(512, latent_dim)
        )

        # For latent-derived fMRI (latent -> fMRI)
        self.latent_to_fmri = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Linear(512, fmri_dim)
        )

        # Decoder input
        self.decoder_input = nn.Linear(latent_dim, 512 * 7 * 7)

        # Decoder to reconstruct image
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def encode_image(self, x):
        vit_outputs = self.encoder(x, output_hidden_states=False)
        hidden_states = vit_outputs.last_hidden_state[:, 0]
        mu = self.fc_mu(hidden_states)
        logvar = self.fc_logvar(hidden_states)
        z_img = self.reparameterize(mu, logvar)
        return z_img, mu, logvar

    def encode_fmri(self, activity):
        return self.fmri_to_latent(activity)

    def decode(self, z):
        recon = self.decoder_input(z)
        recon = recon.view(-1, 512, 7, 7)
        recon = self.decoder(recon)
        return recon

    def forward(self, x, activity):
        # Image -> latent
        z_img, mu, logvar = self.encode_image(x)
        # fMRI -> latent
        z_fmri = self.encode_fmri(activity)
        # latent -> image
        recon = self.decode(z_img)
        # latent -> fMRI
        fmri_pred = self.latent_to_fmri(z_img)
        return recon, mu, logvar, z_img, z_fmri, fmri_pred

def loss_function(recon_x, x, mu, logvar, z_img, z_fmri, fmri_pred, fmri_gt, lambda_fmri=1.0, lambda_fmri_recon=1.0):
    # Image reconstruction loss
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')

    # KL divergence loss
    kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Latent alignment loss: ensure z_img ~ z_fmri
    fmri_latent_loss = F.mse_loss(z_img, z_fmri, reduction='sum')

    # fMRI reconstruction loss: ensure fmri_pred ~ fmri_gt (from image)
    fmri_recon_loss = F.mse_loss(fmri_pred, fmri_gt, reduction='sum')

    return recon_loss + kld_loss + lambda_fmri * fmri_latent_loss + lambda_fmri_recon * fmri_recon_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = VAE(latent_dim=256, fmri_dim=fmri_dim).to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-4)

save_path = '/content/drive/MyDrive/vae_model_fmri'
os.makedirs(save_path, exist_ok=True)

num_epochs = 20
print("Starting joint training!")
for epoch in range(num_epochs):
    vae.train()
    train_loss = 0
    for images, activity in train_loader:
        images = images.to(device)
        activity = activity.to(device)

        optimizer.zero_grad()
        recon_images, mu, logvar, z_img, z_fmri, fmri_pred = vae(images, activity)
        loss = loss_function(recon_images, images, mu, logvar, z_img, z_fmri, fmri_pred, activity)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    avg_loss = train_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")
    torch.save(vae.state_dict(), os.path.join(save_path, f"vae_epoch_{epoch+1}.pth"))

print("Training complete!")


Dataset({
    features: ['split', 'subject', 'subject_id', 'sample_id', 'nsd_id', 'image', 'activity'],
    num_rows: 24450
})
{'split': 'train', 'subject': 'subj01', 'subject_id': 0, 'sample_id': 1352, 'nsd_id': 10459, 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x256 at 0x7FB1AC859B40>, 'activity': [-0.6802130937576294, -0.03223996236920357, -0.28753116726875305, -1.3810890913009644, -0.8335912227630615, -0.9322627782821655, -0.24177655577659607, -0.21463702619075775, 0.1585964858531952, 0.18870893120765686, 0.6244354248046875, 1.6187149286270142, -0.698640763759613, -0.4649065434932709, -1.1286858320236206, -1.2322875261306763, -0.6444418430328369, 0.8209864497184753, -0.5930544137954712, -0.629973292350769, 0.3073829114437103, -0.7263011336326599, -1.1012169122695923, 0.3777778744697571, -0.18045669794082642, -0.2009589821100235, -0.8423824310302734, 0.0539160817861557, -1.6119840145111084, -1.8683416843414307, 0.17024774849414825, -0.6863701343536377, -0.256861

KeyboardInterrupt: 

In [ ]:
import torch
import os
import matplotlib.pyplot as plt

save_path = '/content/drive/MyDrive/vae_model_fmri'
model_file = os.path.join(save_path, 'vae_epoch_5.pth')

fmri_dim = len(train_dataset[0]['activity'])
vae = VAE(latent_dim=256, fmri_dim=fmri_dim).to(device)
vae.load_state_dict(torch.load(model_file, map_location=device))
vae.eval()

images, activity = next(iter(train_loader))
images = images.to(device)

with torch.no_grad():
    # Step 1: Encode image to get z_img, mu, logvar
    vit_outputs = vae.encoder(images, output_hidden_states=False)
    hidden_states = vit_outputs.last_hidden_state[:, 0]
    mu = vae.fc_mu(hidden_states)
    logvar = vae.fc_logvar(hidden_states)
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    z_img = mu + eps * std

    # Step 2: Predict fMRI from z_img
    fmri_pred = vae.latent_to_fmri(z_img)

    # Step 3: Map predicted fMRI back to latent (z_fmri_back)
    z_fmri_back = vae.fmri_to_latent(fmri_pred)

    # Step 4: Decode z_fmri_back to reconstruct the image
    recon_from_fmri = vae.decode(z_fmri_back)

images = images.cpu()
recon_from_fmri = torch.clamp(recon_from_fmri.cpu(), 0, 1)

# Visualization
fig, axes = plt.subplots(2, 8, figsize=(16, 4))
for i in range(8):
    axes[0, i].imshow(images[i].permute(1, 2, 0))
    axes[0, i].axis('off')
    axes[1, i].imshow(recon_from_fmri[i].permute(1, 2, 0))
    axes[1, i].axis('off')

axes[0, 0].set_title('Original Images')
axes[1, 0].set_title('Reconstructed from Predicted fMRI')
plt.show()